In [ ]:
from pynq_dpu import DpuOverlay
from pynq import Overlay
overlay = DpuOverlay("dpu.bit")
import os
import time
import numpy as np
import cv2
import matplotlib.pyplot as plt
from PIL import Image as PIL_Image
from IPython.display import clear_output

overlay.load_model("dpu_tf_inceptionv1.xmodel")


def normalize(image):
    image=image/256.0
    image=image-0.5
    image=image*2
    return image

def preprocess_fn(image):
    image = cv2.resize(image, (224, 224))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = normalize(image)
    return image

def calculate_softmax(data):
    result = np.exp(data)
    return result

def predict_label(softmax):
    with open("img/words.txt", "r") as f:
        lines = f.readlines()
    return lines[np.argmax(softmax)-1]


dpu = overlay.runner
inputTensors = dpu.get_input_tensors()
outputTensors = dpu.get_output_tensors()
shapeIn = tuple(inputTensors[0].dims)
shapeOut = tuple(outputTensors[0].dims)
outputSize = int(outputTensors[0].get_data_size() / shapeIn[0])
softmax = np.empty(outputSize) 
output_data = [np.empty(shapeOut, dtype=np.float32, order="C")]
input_data = [np.empty(shapeIn, dtype=np.float32, order="C")]
image = input_data[0]

video = cv2.VideoCapture(0)


In [ ]:
def single_im():
    _, frame = video.read()
    preprocessed = preprocess_fn(frame)
    image[0,...] = preprocessed.reshape(shapeIn[1:])
    job_id = dpu.execute_async(input_data, output_data)
    dpu.wait(job_id)
    temp = [j.reshape(1, outputSize) for j in output_data]
    softmax = calculate_softmax(temp[0][0])
    confidence = np.amax(softmax)/softmax.sum()
    _, ax = plt.subplots(1)
    _ = ax.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    print("Classification: {}".format(predict_label(softmax)))
    print("Confidence: {}".format(confidence))

In [ ]:
single_im()

In [ ]:
def calculate_fps():
    i = 0
    total = 0
    while True:
        start=time.time()
        
        _, frame = video.read()
        preprocessed = preprocess_fn(frame)
        #end=time.time()
        #start=time.time()
        image[0,...] = preprocessed.reshape(shapeIn[1:])
        job_id = dpu.execute_async(input_data, output_data)
        dpu.wait(job_id)
        temp = [j.reshape(1, outputSize) for j in output_data]
        softmax = calculate_softmax(temp[0][0])
        
        end=time.time()
        total += end-start
        print(i/total)
        i += 1

In [ ]:
calculate_fps()

In [ ]:
def single_im():
    _, frame = video.read()
    preprocessed = preprocess_fn(frame)
    image[0,...] = preprocessed.reshape(shapeIn[1:])
    job_id = dpu.execute_async(input_data, output_data)
    dpu.wait(job_id)
    temp = [j.reshape(1, outputSize) for j in output_data]
    softmax = calculate_softmax(temp[0][0])
    confidence = np.amax(softmax)/softmax.sum()
    _, ax = plt.subplots(1)
    _ = ax.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    print("Classification: {}".format(predict_label(softmax)))
    print("Confidence: {}".format(confidence))

In [ ]:
single_im()

In [ ]:
video.release()
del overlay
del dpu